In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
import os, sys
import pickle
import argparse
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm_notebook as tqdm

import torchvision.models as models
import torch

sys.path.append('../model_opts')
from feature_extraction import *
from model_options import *

In [3]:
from visual_priors import taskonomy_network

In [4]:
model_paths = glob('../model_opts/task_weights/*.pth')
model_names = sorted(['_'.join(model_path.split('.')[0].split('_')[:-1]) for model_path in os.listdir('models') 
                          if 'encoder' in model_path])

In [5]:
image_transforms = get_image_transforms()['imagenet_from_numpy']
mouseland_images = np.load('../stimulus_set.npy')
model_inputs = Variable(torch.stack([image_transforms(img) for img in mouseland_images])).cuda()

In [8]:
problematic_models = ['colorization']
model_names = [name for name in model_names if name not in problematic_models]

In [9]:
output_dir = 'task_rdms'

for model_name in tqdm(model_names): 
    output_file = os.path.join(output_dir, model_name + '_rdms.pkl')
    if not os.path.exists(output_file):
        print('Computing RDMS for {} on {}...'.format(model_name, torch.cuda.get_device_name(0)))
        model = taskonomy_network.TaskonomyEncoder()
        model.load_state_dict(torch.load('models/' + model_name + '_' + 'encoder.pth')['state_dict'])
        model = model.eval()
        model = model.cuda()

        mouseland_feature_maps = get_all_feature_maps(model, model_inputs)

        model_rdms = {}
        for model_layer in tqdm(mouseland_feature_maps, leave=False):
            model_rdms[model_layer] = np.corrcoef(mouseland_feature_maps[model_layer])

        with open(output_file, 'wb') as file:
            pickle.dump(model_rdms, file)

Computing RDMS for curvature on TITAN RTX...


Computing RDMS for denoising on TITAN RTX...


Computing RDMS for depth_euclidean on TITAN RTX...


Computing RDMS for depth_zbuffer on TITAN RTX...


Computing RDMS for edge_occlusion on TITAN RTX...


...

Computing RDMS for room_layout on TITAN RTX...


Computing RDMS for segment_semantic on TITAN RTX...


Computing RDMS for segment_unsup25d on TITAN RTX...


Computing RDMS for segment_unsup2d on TITAN RTX...


Computing RDMS for vanishing_point on TITAN RTX...


In [16]:
output_file = 'taskonomy_zspace_rdms.pkl'

model_rdms = {}
for model_name in tqdm(model_names): 
    if not os.path.exists(output_file):
        print('Computing Latent RDM for {} on {}...'.format(model_name, torch.cuda.get_device_name(0)))
        model = taskonomy_network.TaskonomyEncoder()
        model.load_state_dict(torch.load('models/' + model_name + '_' + 'encoder.pth')['state_dict'])
        model = model.eval()
        model = model.cuda()

        mouseland_feature_maps = get_all_feature_maps(model, model_inputs)
        model_rdms[model_name] = np.corrcoef(mouseland_feature_maps['GroupNorm-1'])

with open(output_file, 'wb') as file:
    pickle.dump(model_rdms, file)

Computing Latent RDM for autoencoding on TITAN RTX...
Computing Latent RDM for class_object on TITAN RTX...
Computing Latent RDM for class_scene on TITAN RTX...
Computing Latent RDM for curvature on TITAN RTX...
Computing Latent RDM for denoising on TITAN RTX...
Computing Latent RDM for depth_euclidean on TITAN RTX...
Computing Latent RDM for depth_zbuffer on TITAN RTX...
Computing Latent RDM for edge_occlusion on TITAN RTX...
Computing Latent RDM for edge_texture on TITAN RTX...
Computing Latent RDM for egomotion on TITAN RTX...
Computing Latent RDM for fixated_pose on TITAN RTX...
Computing Latent RDM for inpainting on TITAN RTX...
Computing Latent RDM for jigsaw on TITAN RTX...
Computing Latent RDM for keypoints2d on TITAN RTX...
Computing Latent RDM for keypoints3d on TITAN RTX...
Computing Latent RDM for nonfixated_pose on TITAN RTX...
Computing Latent RDM for normal on TITAN RTX...
Computing Latent RDM for point_matching on TITAN RTX...
Computing Latent RDM for reshading on TITAN